In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
from fastai import *
from fastai.tabular import *

# Rossmann

## Data preparation

To create the feature-engineered filed train_clean and test_clean from the initial data, run x_009a_rossman_data_clean

In [ ]:
import pyarrow

In [ ]:
PATH = Path('data/rossmann/')
train_df = pd.read_feather(PATH/'train_clean')
test_df = pd.read_feather(PATH/'test_clean')

/home/jhoward/anaconda3/lib/python3.6/site-packages/pandas/io/feather_format.py:112: FutureWarning: `nthreads` argument is deprecated, pass `use_threads` instead
  return feather.read_dataframe(path, nthreads=nthreads)


In [ ]:
train_df.head().T

,0,1,2,3,4
index,0,1,2,3,4
Store,1,2,3,4,5
DayOfWeek,5,5,5,5,5
Date,2015-07-31 00:00:00,2015-07-31 00:00:00,2015-07-31 00:00:00,2015-07-31 00:00:00,2015-07-31 00:00:00
Sales,5263,6064,8314,13995,4822
Customers,555,625,821,1498,559
Open,1,1,1,1,1
Promo,1,1,1,1,1
StateHoliday,False,False,False,False,False
SchoolHoliday,1,1,1,1,1


In [ ]:
cat_vars = ['Store', 'DayOfWeek', 'Year', 'Month', 'Day', 'StateHoliday', 'CompetitionMonthsOpen',
    'Promo2Weeks', 'StoreType', 'Assortment', 'PromoInterval', 'CompetitionOpenSinceYear', 'Promo2SinceYear',
    'State', 'Week', 'Events', 'Promo_fw', 'Promo_bw', 'StateHoliday_fw', 'StateHoliday_bw',
    'SchoolHoliday_fw', 'SchoolHoliday_bw']

cont_vars = ['CompetitionDistance', 'Max_TemperatureC', 'Mean_TemperatureC', 'Min_TemperatureC',
   'Max_Humidity', 'Mean_Humidity', 'Min_Humidity', 'Max_Wind_SpeedKm_h', 
   'Mean_Wind_SpeedKm_h', 'CloudCover', 'trend', 'trend_DE',
   'AfterStateHoliday', 'BeforeStateHoliday', 'Promo', 'SchoolHoliday']

n = len(train_df); n

844338

In [ ]:
idx = np.random.permutation(range(n))[:2000]
idx.sort()
small_train_df = train_df.iloc[idx[:1000]]
small_test_df = train_df.iloc[idx[1000:]]
small_cont_vars = ['CompetitionDistance', 'Mean_Humidity']
small_cat_vars =  ['Store', 'DayOfWeek', 'PromoInterval']
small_train_df = small_train_df[small_cat_vars+small_cont_vars + ['Sales']]
small_test_df = small_test_df[small_cat_vars+small_cont_vars + ['Sales']]

In [ ]:
small_train_df.head()

,Store,DayOfWeek,PromoInterval,CompetitionDistance,Mean_Humidity,Sales
403,405,5,"Jan,Apr,Jul,Oct",1080.0,61,5569
638,640,5,None,8040.0,61,15013
823,825,5,"Jan,Apr,Jul,Oct",380.0,50,6003
890,893,5,"Jan,Apr,Jul,Oct",130.0,50,7179
1558,447,4,None,1510.0,86,4704


In [ ]:
small_test_df.head()

,Store,DayOfWeek,PromoInterval,CompetitionDistance,Mean_Humidity,Sales
428983,379,1,None,6630.0,87,3204
429023,419,1,None,1620.0,78,4121
429633,1033,1,None,7680.0,78,11302
429878,134,6,None,1200.0,78,3582
430509,769,6,"Jan,Apr,Jul,Oct",840.0,77,10220


In [ ]:
categorify = Categorify(small_cat_vars, small_cont_vars)
categorify(small_train_df)
categorify(small_test_df, test=True)

In [ ]:
small_test_df.head()

,Store,DayOfWeek,PromoInterval,CompetitionDistance,Mean_Humidity,Sales
428983,379.0,1,NaN,6630.0,87,3204
429023,419.0,1,NaN,1620.0,78,4121
429633,1033.0,1,NaN,7680.0,78,11302
429878,NaN,6,NaN,1200.0,78,3582
430509,769.0,6,"Jan,Apr,Jul,Oct",840.0,77,10220


In [ ]:
small_train_df['PromoInterval'].cat.codes[:5]

403     1
638    -1
823     1
890     1
1558   -1
dtype: int8

In [ ]:
small_test_df['Store'].cat.codes[:5]

428983    210
429023    233
429633    604
429878     -1
430509    449
dtype: int16

In [ ]:
fill_missing = FillMissing(small_cat_vars, small_cont_vars)
fill_missing(small_train_df)
fill_missing(small_test_df, test=True)

In [ ]:
small_train_df[small_train_df['CompetitionDistance_na'] == True]

In [ ]:
small_test_df[small_test_df['CompetitionDistance_na'] == True]

In [ ]:
train_df = pd.read_feather(PATH/'train_clean')

In [ ]:
idx = np.random.permutation(range(n))[:2000]
idx.sort()
small_train_df = train_df.iloc[idx[:1000]]
small_test_df = train_df.iloc[idx[1000:]]
small_cont_vars = ['CompetitionDistance', 'Mean_Humidity']
small_cat_vars =  ['Store', 'DayOfWeek', 'PromoInterval']
small_train_df = small_train_df[small_cat_vars+small_cont_vars + ['Sales']]
small_test_df = small_test_df[small_cat_vars+small_cont_vars + ['Sales']]

In [ ]:
dep_var = 'Sales'
tfms = [FillMissing, Categorify] #Fillmissing first so that the added columns are categorified
train_ds = TabularDataset.from_dataframe(small_train_df, dep_var, tfms, small_cat_vars, 
                                                    small_cont_vars, log_output=True)
valid_ds = TabularDataset.from_dataframe(small_test_df, dep_var, train_ds.tfms, train_ds.cat_names, 
                                                   train_ds.cont_names, train_ds.stats, log_output=True)

In [ ]:
train_ds[2]

In [ ]:
train_ds.stats, valid_ds.stats

In [ ]:
small_train_df.head()

In [ ]:
cat_vars = ['Store', 'DayOfWeek', 'Year', 'Month', 'Day', 'StateHoliday', 'CompetitionMonthsOpen',
    'Promo2Weeks', 'StoreType', 'Assortment', 'PromoInterval', 'CompetitionOpenSinceYear', 'Promo2SinceYear',
    'State', 'Week', 'Events', 'Promo_fw', 'Promo_bw', 'StateHoliday_fw', 'StateHoliday_bw',
    'SchoolHoliday_fw', 'SchoolHoliday_bw']

cont_vars = ['CompetitionDistance', 'Max_TemperatureC', 'Mean_TemperatureC', 'Min_TemperatureC',
   'Max_Humidity', 'Mean_Humidity', 'Min_Humidity', 'Max_Wind_SpeedKm_h', 
   'Mean_Wind_SpeedKm_h', 'CloudCover', 'trend', 'trend_DE',
   'AfterStateHoliday', 'BeforeStateHoliday', 'Promo', 'SchoolHoliday']

In [ ]:
dep_var = 'Sales'
train_df = pd.read_feather(PATH/'train_clean')
train_df = train_df[cat_vars+cont_vars+[dep_var, 'Date']].copy()

In [ ]:
test_df['Date'].min(), test_df['Date'].max()

In [ ]:
len(test_df)

In [ ]:
cut = train_df['Date'][(train_df['Date'] == train_df['Date'][len(test_df)])].index.max()
cut

In [ ]:
train_df = train_df.set_index('Date')
train_df,valid_df = train_df[cut:], train_df[:cut]
len(train_df),len(valid_df)

In [ ]:
tfms = [FillMissing, Categorify]
data = data_from_tabulardf(PATH, train_df, valid_df, dep_var, tfms=[FillMissing, Categorify], cat_names=cat_vars, 
                           cont_names=cont_vars, log_output=True, num_workers=0)

## Model

In [ ]:
max_log_y = np.log(np.max(train_df['Sales']))
y_range = torch.tensor([0, max_log_y*1.2], device=default_device)

In [ ]:
model = TabularModel(emb_szs, len(cont_vars), 1, [1000,500], [0.001,0.01], emb_drop=0.04, y_range=y_range, is_reg=True)

In [ ]:
model

In [ ]:
learn = Learner(data, model)
learn.loss_fn = F.mse_loss
learn.metrics = [exp_rmspe]

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(5, 1e-3, wd=0.2, pct_start=0.2)

In [ ]:
learn.fit_one_cycle(5, 1e-3, wd=0.1, pct_start=0.3)

In [ ]:
with torch.no_grad():
    pct_var,cnt = 0.,0
    for x,y in learn.data.valid_dl:
        out = learn.model(*x)
        cnt += y.size(0)
        y, out = torch.exp(y), torch.exp(out)
        pct_var += ((y - out)/y).pow(2).sum()
torch.sqrt(pct_var/cnt).item()